In [1]:
from RAG_system_A import RAGSystemA

c:\Users\user1\anaconda3\envs\RAGLLMs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# File paths
index_path = "embeddings/all-MiniLM-L6-v2_faiss_index.index"
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
gemini_token_file = "API_tokens/gemini.txt"

# Initialize the system
rag_system = RAGSystemA(index_path=index_path, embedding_model_name=embedding_model_name, gemini_token_file=gemini_token_file)


RAGSystemA initialized and ready.


In [3]:
# Query the system
user_query = "modernist architecture in Rotterdam"
max_text = 500
top_k = 5

result = rag_system.query(user_query=user_query, max_text=max_text, top_k=top_k)

print("\nLocations Data:")
for location in result["locations"]:
    print(location)

print("\nGenerated Response:")
print(result["response"])

print("\nPerformance Metrics:")
print(f"Retrieval Time: {result['retrieval_time']:.2f} seconds")
print(f"Generation Time: {result['generation_time']:.2f} seconds")
print(f"Total Time: {result['total_time']:.2f} seconds")
print(f"Total Tokens Used: {result['total_tokens']}")
print(f"Prompt Tokens Used: {result['prompt_tokens']}")



Locations Data:
{'title': 'Rotterdam Centrum', 'main_category': 'city', 'subcategories': 'region, unique architecture', 'latitude': np.float64(51.91793055555555), 'longitude': np.float64(4.477872222222223), 'generated_text': "Rotterdam Centrum: A City Reborn\n\nLet's explore the heart of Rotterdam, a city dramatically reshaped by its history.  Its current form is the result of several key events.  Firstly, the city's historical development, culminating in the creation of the Stadsdriehoek, a significant triangular area.  Secondly, the ambitious Water Project, including the construction of the Westersingel canal, significantly altered the city's landscape. Then, the devastating bombing of 1940, which destroyed most of t"}
{'title': 'Pannekoekstraat 46-48', 'main_category': 'building', 'subcategories': 'unique architecture, history', 'latitude': np.float64(51.92348888888888), 'longitude': np.float64(4.4879694444444445), 'generated_text': "This building at 46-48 Pannekoekstraat is more t

In [4]:
# File paths
index_path = "embeddings/all-mpnet-base-v2_faiss_index.index"
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
gemini_token_file = "API_tokens/gemini.txt"

# Initialize the system
rag_system = RAGSystemA(index_path=index_path, embedding_model_name=embedding_model_name, gemini_token_file=gemini_token_file)


RAGSystemA initialized and ready.


In [5]:
# Query the system
user_query = "modernist architecture in Rotterdam"
max_text = 500
top_k = 5

result = rag_system.query(user_query=user_query, max_text=max_text, top_k=top_k)

print("\nLocations Data:")
for location in result["locations"]:
    print(location)

print("\nGenerated Response:")
print(result["response"])

print("\nPerformance Metrics:")
print(f"Retrieval Time: {result['retrieval_time']:.2f} seconds")
print(f"Generation Time: {result['generation_time']:.2f} seconds")
print(f"Total Time: {result['total_time']:.2f} seconds")
print(f"Total Tokens Used: {result['total_tokens']}")
print(f"Prompt Tokens Used: {result['prompt_tokens']}")



Locations Data:
{'title': 'Joliotflat (torenflat)', 'main_category': 'skyscraper', 'subcategories': 'unique architecture, neighbourhood', 'latitude': np.float64(51.96245), 'longitude': np.float64(4.5336694444444445), 'generated_text': "Welcome to the Joliotflat, a prominent high-rise building in the Ommoord district of Rotterdam.  Constructed in 1967, this 62-meter tall building stands as a testament to the architectural style of its time.  With its 20 floors, it's instantly recognizable as one of the many high-rises designed by architect J. Nuts, sharing a similar aesthetic with other buildings in this area. Interestingly, there’s another building nearby that shares the same name, though it's of a different architectural styl"}
{'title': 'Oude Binnenweg 53 Rotterdam', 'main_category': 'building', 'subcategories': 'unique architecture, house', 'latitude': np.float64(51.91789722222222), 'longitude': np.float64(4.476127777777778), 'generated_text': "This building at Oude Binnenweg 53 is